# Inference_demo
This notebook intends to demo how to use the pre-trained fused graph for sementic segmentation

In [1]:
import numpy as np
import tensorflow as tf
from moviepy.editor import VideoFileClip, ImageSequenceClip
import scipy
import matplotlib.pyplot as plt
import os
from glob import glob
from utils import load_image_label_paths, process_raw_data, read_image_label_raw, paste_mask
%load_ext autoreload
%autoreload 2

## Test inference engine for CARLA RAW data

In [2]:
DATA_DIR = './data/Test'
image_paths, label_paths = load_image_label_paths(DATA_DIR,sort=True)
load_pb_path = './inference_pb/fused_graph.pb'
tf.reset_default_graph()
with tf.Session(graph=tf.Graph()) as sess:
    gd = tf.GraphDef()
    with tf.gfile.Open(load_pb_path, 'rb') as f:
        data = f.read()
        gd.ParseFromString(data)
    tf.import_graph_def(gd, name='')
    graph = tf.get_default_graph()
    input_image = graph.get_tensor_by_name('input_image:0')
    softmax = graph.get_tensor_by_name('softmax:0')
    crop_up = 102
    crop_down = 518
    image_shape = [600,800,3]
    results = []
    f = 1
    c_tp = 0
    c_se = 0
    c_re = 0
    r_tp = 0
    r_se = 0
    r_re = 0
    for image_file in image_paths:
        image_raw, label_raw = read_image_label_raw(image_file, label_paths)
        image, label = process_raw_data(image_raw, label_raw, crop=(0,600))
        crop_frame = np.expand_dims(image[crop_up:crop_down,:,:],0)
        out = sess.run(softmax, feed_dict={input_image: crop_frame})[0]
        out_arg = np.argmax(out,axis = 2)
        # Look for red cars :)
        binary_car_result = np.where(out_arg == 1,1,0)
        binary_car_result = np.concatenate((np.zeros([crop_up,image_shape[1]]),binary_car_result), axis=0)
        binary_car_result = np.concatenate((binary_car_result, np.zeros([image_shape[0]-crop_down,image_shape[1]])), axis=0)
        # Look for road :)
        binary_road_result = np.where(out_arg == 2,1,0)
        binary_road_result = np.concatenate((np.zeros([crop_up,image_shape[1]]),binary_road_result), axis=0)
        binary_road_result = np.concatenate((binary_road_result, np.zeros([image_shape[0]-crop_down,image_shape[1]])), axis=0)
        results.append(paste_mask(image, binary_car_result, binary_road_result))
        # Compute Recall and Precision
        c_tp += np.sum(binary_car_result * label[:,:,1])
        c_se += np.sum(binary_car_result)
        c_re += np.sum(label[:,:,1])
        r_tp += np.sum(binary_road_result * label[:,:,2])
        r_se += np.sum(binary_road_result)
        r_re += np.sum(label[:,:,2])
        print('Processing frame: {0}/{1}'.format(f,len(image_paths)),end='\r')
        f +=1
    c_P = c_tp/c_se
    c_R = c_tp/c_re
    r_P = r_tp/r_se
    r_R = r_tp/r_re
    print("Car Precision: {0:.3g} | Car Recall: {1:.3g} | Road Precision: {2:.3g} | Road Recall: {3:.3g} ".format(c_P,c_R,r_P,r_R))

Car Precision: 0.823 | Car Recall: 0.978 | Road Precision: 0.997 | Road Recall: 0.992 


## Convert image sequence to video

In [10]:
output_path = os.path.join('./videos','test.mp4')
output_clip = ImageSequenceClip(results, fps=10)
output_clip.write_videofile(output_path, audio=False)

[MoviePy] >>>> Building video ./videos/test.mp4
[MoviePy] Writing video ./videos/test.mp4


100%|██████████| 1000/1000 [00:09<00:00, 108.37it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./videos/test.mp4 



In [11]:
from IPython.display import HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output_path))